## Resources

- Berserk documentation: https://berserk.readthedocs.io/en/master/index.html

- Lichess API documentation: https://lichess.org/api

- Lichess Database: https://database.lichess.org/

## Installs & Imports

In [ ]:
!pip install berserk
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 737.7 kB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [ ]:
import dotenv
import requests
import os
import berserk
import random
import pandas as pd
import numpy as np
from datetime import date
from tqdm import tqdm
from google.colab import drive
import matplotlib.pyplot as plt

## Load API token and username list

The current username list is retrieved from the game data found on database.lichess.org

In [ ]:
drive.mount('/content/drive')

dotenv.load_dotenv('/content/drive/MyDrive/.env')

api_token = os.environ.get('LICHESS_API_TOKEN')


# import usernames as list:
with open("/content/drive/MyDrive/Colab Notebooks/usernames.txt", "r") as file:
    users = file.read().split(",")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Get a sample of users, start a session

In [ ]:
users_sample = random.sample(users, k=50)
# opening a session to access lichess data:
session = berserk.TokenSession(api_token)
client = berserk.Client(session)

## Get data of sampled users

In [ ]:
help(client.games.export_by_player)

Help on method export_by_player in module berserk.clients.games:

export_by_player(username: 'str', as_pgn: 'bool | None' = None, since: 'int | None' = None, until: 'int | None' = None, max: 'int | None' = None, vs: 'str | None' = None, rated: 'bool | None' = None, perf_type: 'PerfType | None' = None, color: 'Color | None' = None, analysed: 'bool | None' = None, moves: 'bool | None' = None, pgn_in_json: 'bool | None' = None, tags: 'bool | None' = None, clocks: 'bool | None' = None, evals: 'bool | None' = None, opening: 'bool | None' = None, ongoing: 'bool | None' = None, finished: 'bool | None' = None, players: 'str | None' = None, sort: 'str | None' = None, literate: 'bool | None' = None) -> 'Iterator[str] | Iterator[Dict[str, Any]]' method of berserk.clients.games.Games instance
    Get games by player.
    
    :param username: which player's games to return
    :param as_pgn: whether to return the game in PGN format
    :param since: lower bound on the game timestamp
    :param unt

In [ ]:
my_games = pd.DataFrame(list(client.games.export_by_player("oldgandhi", evals=True, clocks=True, analysed=True, max="20", perf_type="rapid")))

In [ ]:
my_games = my_games.drop(labels=["rated","variant","speed"], axis=1)

In [ ]:
my_games.to_csv("/content/drive/MyDrive/Colab Notebooks/evals_clocktimes.csv")

In [ ]:
example_evals = my_games.loc[0,"analysis"]

In [ ]:
[x["eval"] for x in example_evals]

In [ ]:
np.divide(my_games.loc[0,"clocks"],100)

array([900.03, 900.03, 907.39, 905.95, 914.91, 913.79, 920.91, 903.79,
       907.31, 908.91, 842.35, 916.59, 821.55, 921.87, 828.59, 927.55,
       825.63, 931.31, 800.59, 937.07, 763.15, 944.51, 749.23, 949.63,
       756.35, 928.75, 730.75, 885.47, 607.15, 892.91, 537.63, 876.27,
       497.95, 868.27, 444.99, 834.03, 345.79, 784.67, 238.99, 778.99,
       239.07, 784.59, 193.23, 773.47,  56.19, 765.15,  59.87, 771.63,
        56.51, 749.79,  64.35, 740.83,  46.43, 690.27,  40.43, 656.83,
        31.39, 636.27,  34.59, 493.07,  34.67, 465.47,  28.67, 467.71,
        32.35, 470.19,  36.91, 463.95,  46.91, 449.39,  50.27, 419.71,
        36.83, 423.55,  38.75, 423.07,  42.03, 423.08,  39.36])

In [ ]:
# possibly useful if we want to filter out accounts by when they were created
def get_account_age_in_days(start_date: str, end_date: str):
    return abs((date.fromisoformat(start_date) - date.fromisoformat(end_date)).days)

In [ ]:
# rapid information dataframes:
rapid_ratings = pd.DataFrame(columns = ['username', "year", 	"month", 	"day", 	"rating"])
rapid_games = pd.DataFrame(columns = ['username', "games"])

# puzzle information dataframes:
puzzle_ratings = pd.DataFrame(columns = ['username', "year", 	"month", 	"day", 	"rating"])
puzzles = pd.DataFrame(columns = ['username', 'puzzle',])

for user in tqdm(users_sample):
    try:
      user_history = client.users.get_rating_history(user)
      if len(user_history[2]["points"]) > 50: # only extract data if this user has more than 50 rated rapid games.

        # rating data:
        # rapid ratings
        user_rapid_ratings = pd.DataFrame(user_history[2]["points"])
        user_rapid_ratings.insert(0, "username", user)
        user_rapid_ratings['month'] += 1  # because months in lichess API start at 0 we have to increment by 1
        rapid_ratings = pd.concat([rapid_ratings, user_rapid_ratings])

        # puzzle ratings
        user_puzzle_ratings = pd.DataFrame(user_history[13]["points"])
        user_puzzle_ratings.insert(0, "username", user)
        user_puzzle_ratings['month'] += 1  # because months in lichess API start at 0 we have to increment by 1
        puzzle_ratings = pd.concat([puzzle_ratings, user_puzzle_ratings])

        # rapid games information:
        rapid_list = list(client.games.export_by_player(user, perf_type="rapid"))
        rapid_games.loc[len(rapid_games)] = [user, rapid_list]

    except KeyError:
      pass
    except berserk.exceptions.ResponseError:
      pass

100%|██████████| 50/50 [20:41<00:00, 24.83s/it]


## Formatting Rating Data

### Rapid Ratings

In [ ]:
rapid_ratings["date"] = pd.to_datetime(rapid_ratings[rapid_ratings.columns[1:4]])
rapid_ratings = rapid_ratings.drop(labels=["year","month","day"], axis=1)
rapid_ratings

,username,rating,date
0,AnishPanda2016,979,2022-12-13
1,AnishPanda2016,1085,2022-12-14
2,AnishPanda2016,1140,2022-12-16
3,AnishPanda2016,1023,2022-12-17
4,AnishPanda2016,1016,2022-12-18
...,...,...,...
48,thenextbesthang,1917,2023-10-09
49,thenextbesthang,1938,2023-10-25
50,thenextbesthang,1925,2023-10-28
51,thenextbesthang,1924,2023-11-29


### Puzzle Ratings

In [ ]:
puzzle_ratings["date"] = pd.to_datetime(puzzle_ratings[puzzle_ratings.columns[1:4]])
puzzle_ratings = puzzle_ratings.drop(labels=["year","month","day"], axis=1)
puzzle_ratings

,username,rating,date
0,AnishPanda2016,1518,2022-11-17
1,AnishPanda2016,1402,2022-11-21
2,AnishPanda2016,1482,2022-11-22
3,AnishPanda2016,1518,2022-11-23
4,AnishPanda2016,1377,2022-11-27
...,...,...,...
68,thenextbesthang,1757,2023-11-27
69,thenextbesthang,1853,2023-11-28
70,thenextbesthang,1866,2023-11-29
71,thenextbesthang,1828,2023-11-30


### Rapid Games

In [ ]:
# split list of all games for a user into separate rows for each individual game:
rapid_games = rapid_games.explode("games").reset_index(drop=True)
rapid_games

,username,games
0,AnishPanda2016,"{'id': 'gEV7P53H', 'rated': True, 'variant': '..."
1,AnishPanda2016,"{'id': 'rbH7xjlt', 'rated': True, 'variant': '..."
2,AnishPanda2016,"{'id': 'yEfaqwzO', 'rated': True, 'variant': '..."
3,AnishPanda2016,"{'id': 'TlblSEJU', 'rated': True, 'variant': '..."
4,AnishPanda2016,"{'id': 'bgr62XjT', 'rated': True, 'variant': '..."
...,...,...
11979,thenextbesthang,"{'id': 'qyGR8LYh', 'rated': True, 'variant': '..."
11980,thenextbesthang,"{'id': 'dtuGZ6Pz', 'rated': True, 'variant': '..."
11981,thenextbesthang,"{'id': 'muEI15qa', 'rated': True, 'variant': '..."
11982,thenextbesthang,"{'id': 'VILdQjLf', 'rated': True, 'variant': '..."


In [ ]:
# unpack dictionary with game info in column "games" into separate columns:
rapid_games = pd.concat([rapid_games, pd.DataFrame(rapid_games["games"].to_list())], axis=1).drop(labels=["games","speed","tournament","clock"], axis=1)
rapid_games

,username,id,rated,variant,perf,createdAt,lastMoveAt,status,players,winner,moves,swiss,initialFen
0,AnishPanda2016,gEV7P53H,True,standard,rapid,2023-11-26 18:28:34.676000+00:00,2023-11-26 18:41:12.851000+00:00,mate,"{'white': {'user': {'name': 'AnishPanda2016', ...",white,d4 Nc6 Bf4 d6 Nf3 e5 dxe5 dxe5 Qxd8+ Kxd8 Nxe5...,AMrTONKi,NaN
1,AnishPanda2016,rbH7xjlt,True,standard,rapid,2023-11-26 17:55:36.746000+00:00,2023-11-26 18:17:46.865000+00:00,mate,"{'white': {'user': {'name': 'RidhigaA', 'id': ...",black,e4 e6 d4 d5 e5 c5 c3 Nc6 Nf3 Bd7 Bb5 g6 Bxc6 B...,AMrTONKi,NaN
2,AnishPanda2016,yEfaqwzO,True,standard,rapid,2023-11-26 17:44:06.026000+00:00,2023-11-26 17:55:30.791000+00:00,stalemate,"{'white': {'user': {'name': 'AnishPanda2016', ...",NaN,d4 d5 Bf4 Bf5 Nf3 Nc6 e3 Nf6 c3 e6 Bd3 Bg6 Qc2...,AMrTONKi,NaN
3,AnishPanda2016,TlblSEJU,True,standard,rapid,2023-11-26 17:26:19.476000+00:00,2023-11-26 17:40:13.812000+00:00,draw,"{'white': {'user': {'name': 'virajseshia', 'id...",NaN,e4 e6 Nc3 d5 exd5 exd5 Nf3 Nf6 d3 Nc6 d4 Bb4 B...,AMrTONKi,NaN
4,AnishPanda2016,bgr62XjT,True,standard,rapid,2023-11-26 17:00:02.501000+00:00,2023-11-26 17:08:59.565000+00:00,mate,"{'white': {'user': {'name': 'AnishPanda2016', ...",white,d4 d5 Bf4 Nc6 Nf3 Nf6 e3 Bg4 c3 Bxf3 Qxf3 e6 B...,AMrTONKi,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11979,thenextbesthang,qyGR8LYh,True,standard,rapid,2021-07-22 20:06:59.034000+00:00,2021-07-22 20:23:16.968000+00:00,resign,"{'white': {'user': {'name': 'thenextbesthang',...",white,e4 e5 Bc4 Nc6 Nf3 d6 O-O h6 d4 exd4 Nxd4 Nxd4 ...,NaN,NaN
11980,thenextbesthang,dtuGZ6Pz,True,standard,rapid,2021-07-22 19:36:13.584000+00:00,2021-07-22 19:45:17.189000+00:00,resign,"{'white': {'user': {'name': 'thenextbesthang',...",black,e4 e5 Nf3 Nc6 Bc4 h6 Nc3 Nf6 d3 Bc5 Be3 Bxe3 f...,NaN,NaN
11981,thenextbesthang,muEI15qa,True,standard,rapid,2021-07-22 16:28:16.820000+00:00,2021-07-22 16:35:04.238000+00:00,mate,"{'white': {'user': {'name': 'RenzoZagarra', 'i...",white,e4 e5 Nc3 Nc6 Bc4 Nf6 d3 Bc5 f4 d6 Nf3 O-O Na4...,NaN,NaN
11982,thenextbesthang,VILdQjLf,True,standard,rapid,2021-07-22 16:25:24.370000+00:00,2021-07-22 16:27:56.254000+00:00,resign,"{'white': {'user': {'name': 'thenextbesthang',...",white,e4 e5 Nf3 Nc6 Bc4 Nf6 d3 Bc5 Be3 Bxe3 fxe3 Ng4...,NaN,NaN


In [ ]:
# drop games with nonstandard starting position:
rapid_games = rapid_games[rapid_games["variant"]=="standard"]

# change id to link:
rapid_games["id"] = "https://lichess.org/" + rapid_games["id"]

<ipython-input-9-77f20675bd9b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rapid_games["id"] = "https://lichess.org/" + rapid_games["id"]


In [ ]:
rapid_games = rapid_games.reset_index(drop=True)
rapid_games

,username,id,rated,variant,perf,createdAt,lastMoveAt,status,players,winner,moves,swiss,initialFen
0,AnishPanda2016,https://lichess.org/gEV7P53H,True,standard,rapid,2023-11-26 18:28:34.676000+00:00,2023-11-26 18:41:12.851000+00:00,mate,"{'white': {'user': {'name': 'AnishPanda2016', ...",white,d4 Nc6 Bf4 d6 Nf3 e5 dxe5 dxe5 Qxd8+ Kxd8 Nxe5...,AMrTONKi,NaN
1,AnishPanda2016,https://lichess.org/rbH7xjlt,True,standard,rapid,2023-11-26 17:55:36.746000+00:00,2023-11-26 18:17:46.865000+00:00,mate,"{'white': {'user': {'name': 'RidhigaA', 'id': ...",black,e4 e6 d4 d5 e5 c5 c3 Nc6 Nf3 Bd7 Bb5 g6 Bxc6 B...,AMrTONKi,NaN
2,AnishPanda2016,https://lichess.org/yEfaqwzO,True,standard,rapid,2023-11-26 17:44:06.026000+00:00,2023-11-26 17:55:30.791000+00:00,stalemate,"{'white': {'user': {'name': 'AnishPanda2016', ...",NaN,d4 d5 Bf4 Bf5 Nf3 Nc6 e3 Nf6 c3 e6 Bd3 Bg6 Qc2...,AMrTONKi,NaN
3,AnishPanda2016,https://lichess.org/TlblSEJU,True,standard,rapid,2023-11-26 17:26:19.476000+00:00,2023-11-26 17:40:13.812000+00:00,draw,"{'white': {'user': {'name': 'virajseshia', 'id...",NaN,e4 e6 Nc3 d5 exd5 exd5 Nf3 Nf6 d3 Nc6 d4 Bb4 B...,AMrTONKi,NaN
4,AnishPanda2016,https://lichess.org/bgr62XjT,True,standard,rapid,2023-11-26 17:00:02.501000+00:00,2023-11-26 17:08:59.565000+00:00,mate,"{'white': {'user': {'name': 'AnishPanda2016', ...",white,d4 d5 Bf4 Nc6 Nf3 Nf6 e3 Bg4 c3 Bxf3 Qxf3 e6 B...,AMrTONKi,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11884,thenextbesthang,https://lichess.org/qyGR8LYh,True,standard,rapid,2021-07-22 20:06:59.034000+00:00,2021-07-22 20:23:16.968000+00:00,resign,"{'white': {'user': {'name': 'thenextbesthang',...",white,e4 e5 Bc4 Nc6 Nf3 d6 O-O h6 d4 exd4 Nxd4 Nxd4 ...,NaN,NaN
11885,thenextbesthang,https://lichess.org/dtuGZ6Pz,True,standard,rapid,2021-07-22 19:36:13.584000+00:00,2021-07-22 19:45:17.189000+00:00,resign,"{'white': {'user': {'name': 'thenextbesthang',...",black,e4 e5 Nf3 Nc6 Bc4 h6 Nc3 Nf6 d3 Bc5 Be3 Bxe3 f...,NaN,NaN
11886,thenextbesthang,https://lichess.org/muEI15qa,True,standard,rapid,2021-07-22 16:28:16.820000+00:00,2021-07-22 16:35:04.238000+00:00,mate,"{'white': {'user': {'name': 'RenzoZagarra', 'i...",white,e4 e5 Nc3 Nc6 Bc4 Nf6 d3 Bc5 f4 d6 Nf3 O-O Na4...,NaN,NaN
11887,thenextbesthang,https://lichess.org/VILdQjLf,True,standard,rapid,2021-07-22 16:25:24.370000+00:00,2021-07-22 16:27:56.254000+00:00,resign,"{'white': {'user': {'name': 'thenextbesthang',...",white,e4 e5 Nf3 Nc6 Bc4 Nf6 d3 Bc5 Be3 Bxe3 fxe3 Ng4...,NaN,NaN


In [ ]:
# extract white and black columns from players column:
players = pd.DataFrame(rapid_games["players"].to_list())
players = players.applymap(lambda x: x.get("user", {}).get("name"))
rapid_games = pd.concat([rapid_games, players], axis=1)

In [ ]:
# drop some more of the unnecessary columns
rapid_games = rapid_games.drop(labels=["variant","perf","players","swiss","initialFen"], axis=1)

In [ ]:
rapid_games

,username,id,rated,createdAt,lastMoveAt,status,winner,moves,white,black
0,AnishPanda2016,https://lichess.org/gEV7P53H,True,2023-11-26 18:28:34.676000+00:00,2023-11-26 18:41:12.851000+00:00,mate,white,d4 Nc6 Bf4 d6 Nf3 e5 dxe5 dxe5 Qxd8+ Kxd8 Nxe5...,AnishPanda2016,Kanishkarr
1,AnishPanda2016,https://lichess.org/rbH7xjlt,True,2023-11-26 17:55:36.746000+00:00,2023-11-26 18:17:46.865000+00:00,mate,black,e4 e6 d4 d5 e5 c5 c3 Nc6 Nf3 Bd7 Bb5 g6 Bxc6 B...,RidhigaA,AnishPanda2016
2,AnishPanda2016,https://lichess.org/yEfaqwzO,True,2023-11-26 17:44:06.026000+00:00,2023-11-26 17:55:30.791000+00:00,stalemate,NaN,d4 d5 Bf4 Bf5 Nf3 Nc6 e3 Nf6 c3 e6 Bd3 Bg6 Qc2...,AnishPanda2016,AmazingRithvik
3,AnishPanda2016,https://lichess.org/TlblSEJU,True,2023-11-26 17:26:19.476000+00:00,2023-11-26 17:40:13.812000+00:00,draw,NaN,e4 e6 Nc3 d5 exd5 exd5 Nf3 Nf6 d3 Nc6 d4 Bb4 B...,virajseshia,AnishPanda2016
4,AnishPanda2016,https://lichess.org/bgr62XjT,True,2023-11-26 17:00:02.501000+00:00,2023-11-26 17:08:59.565000+00:00,mate,white,d4 d5 Bf4 Nc6 Nf3 Nf6 e3 Bg4 c3 Bxf3 Qxf3 e6 B...,AnishPanda2016,Nilani
...,...,...,...,...,...,...,...,...,...,...
11884,thenextbesthang,https://lichess.org/qyGR8LYh,True,2021-07-22 20:06:59.034000+00:00,2021-07-22 20:23:16.968000+00:00,resign,white,e4 e5 Bc4 Nc6 Nf3 d6 O-O h6 d4 exd4 Nxd4 Nxd4 ...,thenextbesthang,Ryzvelts
11885,thenextbesthang,https://lichess.org/dtuGZ6Pz,True,2021-07-22 19:36:13.584000+00:00,2021-07-22 19:45:17.189000+00:00,resign,black,e4 e5 Nf3 Nc6 Bc4 h6 Nc3 Nf6 d3 Bc5 Be3 Bxe3 f...,thenextbesthang,Inamul
11886,thenextbesthang,https://lichess.org/muEI15qa,True,2021-07-22 16:28:16.820000+00:00,2021-07-22 16:35:04.238000+00:00,mate,white,e4 e5 Nc3 Nc6 Bc4 Nf6 d3 Bc5 f4 d6 Nf3 O-O Na4...,RenzoZagarra,thenextbesthang
11887,thenextbesthang,https://lichess.org/VILdQjLf,True,2021-07-22 16:25:24.370000+00:00,2021-07-22 16:27:56.254000+00:00,resign,white,e4 e5 Nf3 Nc6 Bc4 Nf6 d3 Bc5 Be3 Bxe3 fxe3 Ng4...,thenextbesthang,Dan_Wulff


In [ ]:
# add outcome of the game (win, draw, loss) from the perspective of the username:
conditions = [
    rapid_games["winner"].isna(),
    (rapid_games["winner"]=="white") & (rapid_games["white"]==rapid_games["username"]),
    (rapid_games["winner"]=="black") & (rapid_games["black"]==rapid_games["username"]),
]
choices = [
    "draw",
    "win",
    "win"
]
rapid_games["outcome"] = np.select(conditions, choices, default="loss")

In [ ]:
rapid_games.value_counts("username") # number of games per user

username
AZ_ultra1901       4751
Hilson7            1584
elgalgo72           907
Thinkersaurish      800
ChessCheens         524
AnishPanda2016      486
JeanEric            382
dyllos              381
Adrian_Glez         346
te989               330
JeremyMNunez        322
edahhani            239
Sarmiflex           208
Elcachoras12        202
SamuelTorrres       155
Melag               147
thenextbesthang     125
dtype: int64

In [ ]:
rapid_games # final game data set

,username,id,rated,createdAt,lastMoveAt,status,winner,moves,white,black,outcome
0,AnishPanda2016,https://lichess.org/gEV7P53H,True,2023-11-26 18:28:34.676000+00:00,2023-11-26 18:41:12.851000+00:00,mate,white,d4 Nc6 Bf4 d6 Nf3 e5 dxe5 dxe5 Qxd8+ Kxd8 Nxe5...,AnishPanda2016,Kanishkarr,win
1,AnishPanda2016,https://lichess.org/rbH7xjlt,True,2023-11-26 17:55:36.746000+00:00,2023-11-26 18:17:46.865000+00:00,mate,black,e4 e6 d4 d5 e5 c5 c3 Nc6 Nf3 Bd7 Bb5 g6 Bxc6 B...,RidhigaA,AnishPanda2016,win
2,AnishPanda2016,https://lichess.org/yEfaqwzO,True,2023-11-26 17:44:06.026000+00:00,2023-11-26 17:55:30.791000+00:00,stalemate,NaN,d4 d5 Bf4 Bf5 Nf3 Nc6 e3 Nf6 c3 e6 Bd3 Bg6 Qc2...,AnishPanda2016,AmazingRithvik,draw
3,AnishPanda2016,https://lichess.org/TlblSEJU,True,2023-11-26 17:26:19.476000+00:00,2023-11-26 17:40:13.812000+00:00,draw,NaN,e4 e6 Nc3 d5 exd5 exd5 Nf3 Nf6 d3 Nc6 d4 Bb4 B...,virajseshia,AnishPanda2016,draw
4,AnishPanda2016,https://lichess.org/bgr62XjT,True,2023-11-26 17:00:02.501000+00:00,2023-11-26 17:08:59.565000+00:00,mate,white,d4 d5 Bf4 Nc6 Nf3 Nf6 e3 Bg4 c3 Bxf3 Qxf3 e6 B...,AnishPanda2016,Nilani,win
...,...,...,...,...,...,...,...,...,...,...,...
11884,thenextbesthang,https://lichess.org/qyGR8LYh,True,2021-07-22 20:06:59.034000+00:00,2021-07-22 20:23:16.968000+00:00,resign,white,e4 e5 Bc4 Nc6 Nf3 d6 O-O h6 d4 exd4 Nxd4 Nxd4 ...,thenextbesthang,Ryzvelts,win
11885,thenextbesthang,https://lichess.org/dtuGZ6Pz,True,2021-07-22 19:36:13.584000+00:00,2021-07-22 19:45:17.189000+00:00,resign,black,e4 e5 Nf3 Nc6 Bc4 h6 Nc3 Nf6 d3 Bc5 Be3 Bxe3 f...,thenextbesthang,Inamul,loss
11886,thenextbesthang,https://lichess.org/muEI15qa,True,2021-07-22 16:28:16.820000+00:00,2021-07-22 16:35:04.238000+00:00,mate,white,e4 e5 Nc3 Nc6 Bc4 Nf6 d3 Bc5 f4 d6 Nf3 O-O Na4...,RenzoZagarra,thenextbesthang,loss
11887,thenextbesthang,https://lichess.org/VILdQjLf,True,2021-07-22 16:25:24.370000+00:00,2021-07-22 16:27:56.254000+00:00,resign,white,e4 e5 Nf3 Nc6 Bc4 Nf6 d3 Bc5 Be3 Bxe3 fxe3 Ng4...,thenextbesthang,Dan_Wulff,win


## Save as .csv

In [ ]:
puzzle_ratings.to_csv("/content/drive/MyDrive/Colab Notebooks/puzzle_ratings.csv")
rapid_ratings.to_csv("/content/drive/MyDrive/Colab Notebooks/rapid_ratings.csv")
rapid_games.to_csv("/content/drive/MyDrive/Colab Notebooks/rapid_games.csv")